In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from decouple import config

import os


In [2]:
load_dotenv("/home/longview/Dev/.env")

True

In [3]:
llm = ChatOpenAI()

In [9]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing language detection and translation capabilities. This can be useful for testing applications and websites in multiple languages to ensure they are functioning correctly for users around the world. Langsmith can also assist with testing the accuracy and effectiveness of language processing algorithms and models. Additionally, Langsmith can help with localization testing to ensure that content is culturally appropriate and linguistically accurate for different target markets.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 15, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}, id='run-aeafce5f-8dd1-4384-85f9-0c6bd51977f9-0')

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [11]:
chain = prompt | llm

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can be a valuable tool for testing in various ways. Here are some ways in which Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to write automated test scripts for testing software applications. By using Langsmith to automate test cases, you can increase test coverage, reduce manual effort, and improve testing efficiency.\n\n2. Data Generation: Langsmith can be used to generate test data for different scenarios and inputs. This can help in creating diverse test cases and ensuring thorough testing of the application.\n\n3. Integration Testing: Langsmith can be used to automate integration testing by simulating interactions between different components or systems. This can help in identifying issues related to data exchange, communication protocols, and system integration.\n\n4. Performance Testing: Langsmith can be used to simulate a large number of virtual users to test the performance of an application under different load c

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
chain = prompt | llm | output_parser

In [15]:
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can help with testing in various ways:\n\n1. **Automated Testing**: Langsmith can generate test cases automatically based on the specifications provided. This can help in quickly creating a comprehensive set of test cases to cover various scenarios.\n\n2. **Test Data Generation**: Langsmith can generate test data that can be used to test the application under different conditions. This can help in testing the application's robustness and performance.\n\n3. **Code Analysis**: Langsmith can analyze the codebase and identify potential issues such as code smells, bugs, or security vulnerabilities. This can help in improving the quality of the code and reduce the chances of bugs slipping through testing.\n\n4. **Performance Testing**: Langsmith can simulate different load conditions and analyze the application's performance under stress. This can help in identifying performance bottlenecks and optimizing the application for better performance.\n\n5. **Regression Testing**: Langsm

## Retrieval Chain

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          
                                          <context>
                                          {context}
                                          </context>

                                          Question: {input}"""
                                          )

document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, and run custom evaluations to score test results. Additionally, LangSmith provides a comparison view to track and diagnose regressions in test scores across multiple revisions of an application. The platform also offers a playground environment for rapid iteration and experimentation, as well as features for beta testing, capturing feedback, annotating traces, and adding runs to datasets to refine and improve application performance.


## Conversation Retrieval Chain

In [25]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# We first need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [26]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [28]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug whe

## Agent

In [29]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)